In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")
install_package("cloudinary")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")

# Entering the APIs to be Sotred

In [ ]:
pip install cloudinary


In [ ]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")

# Function to create/update .env file with Cloudinary API keys
def set_cloudinary_api_keys():
    # Ask user for the Cloudinary credentials without displaying the input
    cloud_name = input("Please enter your Cloudinary Cloud Name: ")
    api_key = getpass.getpass("Please enter your Cloudinary API key: ")
    api_secret = getpass.getpass("Please enter your Cloudinary API secret: ")

    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the keys; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"CLOUDINARY_CLOUD_NAME={cloud_name}\n")
            file.write(f"CLOUDINARY_API_KEY={api_key}\n")
            file.write(f"CLOUDINARY_API_SECRET={api_secret}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"CLOUDINARY_CLOUD_NAME={cloud_name}\n")
            file.write(f"CLOUDINARY_API_KEY={api_key}\n")
            file.write(f"CLOUDINARY_API_SECRET={api_secret}\n")
    
    print(f"Cloudinary API keys added to {env_file}")

# Call the functions to set the API key and credentials
set_ngrok_api_key()
set_cloudinary_api_keys()


# Initialize Cloudinary with credentials from environment variables
import cloudinary
from cloudinary import config

cloudinary.config(
    cloud_name=os.getenv('CLOUDINARY_CLOUD_NAME'),
    api_key=os.getenv('CLOUDINARY_API_KEY'),
    api_secret=os.getenv('CLOUDINARY_API_SECRET')
)

# The .env file should now have these values:
# CLOUDINARY_CLOUD_NAME=your_cloud_name
# CLOUDINARY_API_KEY=your_api_key
# CLOUDINARY_API_SECRET=your_api_secret
# NGROK_AUTHTOKEN=your_ngrok_auth_token


# [](http://)[](http://)******Fucntions for the Operations of CNN

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt
import requests
import cloudinary
import cloudinary.uploader

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Initialize Cloudinary with credentials from environment variables
print(os.getenv('CLOUDINARY_CLOUD_NAME'))
cloudinary.config(
    cloud_name=os.getenv('CLOUDINARY_CLOUD_NAME'),
    api_key=os.getenv('CLOUDINARY_API_KEY'),
    api_secret=os.getenv('CLOUDINARY_API_SECRET')
)

# Additional Kernels
kernels = {
    "Vertical Edge": np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    "Horizontal Edge": np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    "Smoothing": np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9,
    "Gaussian Blur": np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16,
    "Emboss": np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
}

def apply_kernels_with_multiple_layers(images, kernels_list):
    """
    Apply multiple convolutional layers with different kernels sequentially to the images.
    """
    input_layer = Input(shape=(500, 500, 1))
    x = input_layer

    # Add convolutional layers for each kernel in the list
    for kernel in kernels_list:
        kernel = kernel[..., np.newaxis, np.newaxis]  # Adjust kernel dimensions
        x = Conv2D(
            filters=1,
            kernel_size=(3, 3),
            kernel_initializer=tf.keras.initializers.Constant(kernel),
            use_bias=False,
            padding='same'
        )(x)

    # Build and compile the model
    model = Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='mse')

    # Predict and return the output
    return model.predict(images)


def load_and_process_image(image_content):
    """Load the image from bytes and process it."""
    try:
        # Load image using OpenCV
        img = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Invalid image data")

        # Resize image
        img = cv2.resize(img, (500, 500))
        img = img[np.newaxis, ..., np.newaxis] / 255.0  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error loading and processing image: {e}")
        return None

# Function to upload images to Cloudinary
def upload_to_cloudinary(image_path):
    """Uploads the image to Cloudinary and returns the image URL."""
    try:
        response = cloudinary.uploader.upload(image_path)
        return response['secure_url']
    except Exception as e:
        print(f"Error uploading image to Cloudinary: {e}")
        return None

@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']
        layers = data.get('layers', 2)  # Default to 2 layers if not provided

        # Ensure 'layers' is a valid integer greater than 0
        if not isinstance(layers, int) or layers <= 0:
            return jsonify({"error": "'layers' must be a positive integer"}), 400

        # Download the image
        print(f"Downloading file from: {file_url}")
        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Load and process the image
        image = load_and_process_image(file_response.content)
        if image is None:
            return jsonify({"error": "Failed to load image"}), 400

        # Apply kernels with multiple convolutional layers
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)
        results = {}

        for name, kernel in kernels.items():
            # print(f"Applying multiple layers with {name} kernel...")

            # Use the layers parameter instead of hardcoded 3
            convolved_image = apply_kernels_with_multiple_layers(image, [kernel] * layers)
            
            # Debugging: Check image shape after convolution
            print(f"Convolved image for kernel {name}: {convolved_image.shape}")

            # Save image as PNG file
            output_path = os.path.join(output_dir, f"{name}_result.png")
            plt.imsave(output_path, convolved_image.squeeze(), cmap='gray')
            
            # Upload the image to Cloudinary and get the URL
            cloudinary_url = upload_to_cloudinary(output_path)

            # Debugging: Check if the image was uploaded successfully
            if cloudinary_url:
                # print(f"Uploaded image to Cloudinary: {cloudinary_url}")
                results[name] = {
                    "image_url": cloudinary_url,
                    "kernelName": name,
                    "isMultipleLayers": True,
                    "layersApplied": layers  # Store the number of layers applied
                }
            else:
                print(f"Failed to upload image for kernel {name}")

        if not results:
            print("No results found after processing.")

        print("Image processing completed.")
        return jsonify({
            "message": "Image processing completed",
            "results": results
        }), 200

    except Exception as e:
        print(f"Error processing image: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/testl', methods=['GET'])
def test():
    print("Ok API Gateway working fine")


# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)


# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url


# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Start ngrok and print the URL
public_url = run_ngrok()

# Keep the program running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Server shutting down.")


# Implementation with Multiple CNN layer and 6 Kernels

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt
import requests
import base64
import io

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)



# Additional Kernels
kernels = {
    "Vertical Edge": np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    "Horizontal Edge": np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    "Smoothing": np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9,
    "Gaussian Blur": np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16,
    "Emboss": np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
}

def apply_kernels_with_multiple_layers(images, kernels_list):
    """
    Apply multiple convolutional layers with different kernels sequentially to the images.
    """
    input_layer = Input(shape=(500, 500, 1))
    x = input_layer

    # Add convolutional layers for each kernel in the list
    for kernel in kernels_list:
        kernel = kernel[..., np.newaxis, np.newaxis]  # Adjust kernel dimensions
        x = Conv2D(
            filters=1,
            kernel_size=(3, 3),
            kernel_initializer=tf.keras.initializers.Constant(kernel),
            use_bias=False,
            padding='same'
        )(x)

    # Build and compile the model
    model = Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='mse')

    # Predict and return the output
    return model.predict(images)


def load_and_process_image(image_content):
    """Load the image from bytes and process it."""
    try:
        # Load image using OpenCV
        img = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Invalid image data")

        # Resize image
        img = cv2.resize(img, (500, 500))
        img = img[np.newaxis, ..., np.newaxis] / 255.0  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error loading and processing image: {e}")
        return None



@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Download the image
        print(f"Downloading file from: {file_url}")
        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Load and process the image
        image = load_and_process_image(file_response.content)
        if image is None:
            return jsonify({"error": "Failed to load image"}), 400

        # Apply kernels with multiple convolutional layers
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)
        results = {}

        for name, kernel in kernels.items():
            print(f"Applying multiple layers with {name} kernel...")
            convolved_image = apply_kernels_with_multiple_layers(image, [kernel] * 3)  # Using the same kernel for two layers
            
            # Save image as PNG file
            output_path = os.path.join(output_dir, f"{name}_result.png")
            plt.imsave(output_path, convolved_image.squeeze(), cmap='gray')
            
            # Read image and encode it to base64
            with open(output_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

            # Store results with the image in base64 and additional metadata
            results[name] = {
                "image": img_base64,
                "kernelName": name,
                "isMultipleLayers": True
            }

        print("Image processing completed.")
        return jsonify({
            "message": "Image processing completed",
            "results": results
        }), 200

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500



# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)


# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url


# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Start ngrok and print the URL
public_url = run_ngrok()

# Keep the program running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Server shutting down.")


# Implementation with Single CNN layer and 4 Kernels

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt
import requests
import base64

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Kernels for convolution
kernels = {
    "Vertical Edge": np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    "Horizontal Edge": np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    "Smoothing": np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9
}

# Test endpoint to check if the server is running
@app.route('/test', methods=['POST'])
def test():
    return jsonify({"message": "Flask server is running!"})


def load_and_process_image(image_content):
    """Load the image from bytes and process it."""
    try:
        # Load image using OpenCV
        img = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Invalid image data")

        # Resize image
        img = cv2.resize(img, (500, 500))
        img = img[np.newaxis, ..., np.newaxis] / 255.0  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error loading and processing image: {e}")
        return None


def apply_kernel(images, kernel):
    """Apply the given kernel to the image."""
    kernel = kernel[..., np.newaxis, np.newaxis]
    input_layer = Input(shape=(500, 500, 1))
    conv_layer = Conv2D(
        filters=1,
        kernel_size=(3, 3),
        kernel_initializer=tf.keras.initializers.Constant(kernel),
        use_bias=False,
        padding='same'
    )(input_layer)
    model = Model(inputs=input_layer, outputs=conv_layer)
    model.compile(optimizer='adam', loss='mse')
    return model.predict(images)


@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Download the image
        print(f"Downloading file from: {file_url}")
        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Load and process the image
        image = load_and_process_image(file_response.content)
        if image is None:
            return jsonify({"error": "Failed to load image"}), 400

        # Apply kernels and save results
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)
        results = {}
        for name, kernel in kernels.items():
            print(f"Applying {name} kernel...")
            convolved_image = apply_kernel(image, kernel)
            output_path = os.path.join(output_dir, f"{name}_result.png")
            plt.imsave(output_path, convolved_image.squeeze(), cmap='gray')

            # Read image and encode it to base64
            with open(output_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

            # Store results with the image in base64 and additional metadata
            results[name] = {
                "image": img_base64,
                "kernelName": name,
                "isMultipleLayers": False  # Added flag for single-layer kernel
            }

        print("Image processing completed.")
        return jsonify({
            "message": "Image processing completed",
            "results": results
        }), 200

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500


# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)


# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url


# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Start ngrok and print the URL
public_url = run_ngrok()

# Keep the program running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Server shutting down.")
